In [832]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer

In [833]:
def distance_into_probability(x, w, b):
    ''' Функция возвращает по найденному уравнению разделяющей плоскости вероятность объекта быть идеальным
    x - это признаки данного объекта
    b - вектор коэффициентов в уравнении плоскости '''
    lin_comb = b
    for i in range(len(x)):
        lin_comb += x[i] * w[i]
    return 1 / (1 + np.exp(-lin_comb))

In [834]:
def get_probabilty(model, row):
    ''' Функция возвращает по найденному уравнению разделяющей плоскости вероятность объекта быть идеальным
    x - это признаки данного объекта
    b - вектор коэффициентов в уравнении плоскости '''
    des = model.decision_function([row])
    return 1 / (1 + np.exp(-des[0]))

Делим исходные данные на признаки и ответы, а также разделяем на данные, на которых обучаем и на которых тестируем

In [835]:
df = pd.read_csv('../assets/train.csv')

X = df.loc[:, :'pollutedPointsPercentage']
y = df.loc[:, 'result']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [836]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

Объявление и обучение модели

In [837]:
model = svm.SVC(kernel='linear', C = 1.0, probability=True)
model.fit(x_train, y_train)

SVC(kernel='linear', probability=True)

In [838]:
get_probabilty(model, x_test[10]) * 100

0.13868135232510675

Экспортирование модели в папку `/models`

In [839]:
import joblib
joblib.dump(model, '../models/svc.pkl')

['../models/svc.pkl']